# Quadratic Programming (QP)

## What is Quadratic Programming?
A **Quadratic Program (QP)** is an optimization problem where:
- the **objective function is quadratic**
- the **constraints are linear**

In other words:
> **QP = LP + a quadratic objective**

---

## Standard QP Formulation

A common formulation (used in MATLAB, Pyomo solvers, etc.) is:

$\min \left( \frac{1}{2} x^T H x + f^T x \right)$

---

## Components Explained

### Decision Variables
$
x =
\begin{bmatrix}
x_1 \\
x_2 \\
\vdots \\
x_n
\end{bmatrix}
$

This vector contains all decision variables of the problem.

---

### Quadratic Term (Curvature)
$
H =
\begin{bmatrix}
h_{11} & h_{12} \\
h_{21} & h_{22}
\end{bmatrix}
$

- $H$ is a **constant matrix** defining the quadratic coefficients
- It determines how strongly large values of \(x\) are penalized
- If $H$ is **positive semidefinite**, the problem is **convex** (good news for solvers)

The quadratic expression:  
$ x^T H x $  
produces squared terms like $x_1^2, x_2^2$, and cross-terms like $x_1 x_2$.

The factor $\frac{1}{2}$ is included by convention to simplify derivatives.

---

### Linear Term
$
f =
\begin{bmatrix}
f_1 \\
f_2 \\
\vdots \\
f_n
\end{bmatrix}
$

- This is the same linear term used in Linear Programming
- It shifts the minimum of the quadratic function

---

## Constraints

All constraints in QP are **linear**.

### Variable Bounds
$ LB \leq x \leq UB $

- $LB$ and $UB$ are vectors of lower and upper bounds
- Each entry corresponds to one decision variable

---

### Equality Constraints
$ A_{eq} x = b_{eq} $

- $A_{eq}$: matrix of size $n_{eq} \times n$
- $b_{eq}$: vector of size $n_{eq} \times 1$
- Each row represents one equality constraint

---

### Inequality Constraints
$ A_{ineq} x \leq b_{ineq} $

- $A_{ineq}$: matrix of size $n_{ineq} \times n$
- $b_{ineq}$: vector of size $n_{ineq} \times 1$
- Each row represents one inequality constraint

---

## Intuition

- **LP** → flat objective (straight line / plane)
- **QP** → curved objective (bowl-shaped)
- Constraints stay linear
- The quadratic term discourages extreme solutions and encourages smoothness

---
## <b style="color:red">IMPORTANT NOTE</b>
The matrix formulation shown above is used internally by many solvers (and commonly shown in MATLAB-style interfaces).

When modeling problems in **Pyomo**, constraints and objectives are written **directly as equations**, not in matrix form.  
Pyomo automatically converts these expressions into the appropriate matrix representation before passing them to the solver.

The matrix form is presented here **for explanation purposes only**, to help understand how QP solvers work under the hood.

## TL;DR
Quadratic Programming is **Linear Programming written in matrix form with a quadratic objective**:

$\min \left( \frac{1}{2} x^T H x + f^T x \right)$

Same variables.  
Same constraints.  
Just a smoother objective.

--- 
## Production Cost Optimization Problem
### Problem Description
A manufacturer has developed an empirical model of its production system and has determined that the daily production cost is given by:

$\text{cost} = 0.4x_1^2 - 5x_1 + x_2^2 - 6x_2 + 50$

where $x_1$ and $x_2$ are the decision variables representing the number of hours per day that Unit 1 and Unit 2 are operated, respectively. The objective is to determine the optimal operating hours for both units that minimize the total production cost.

The manufacturer must produce at least 20,000 units per day.  
Unit 1 is an older but durable machine that produces 2,700 units per hour and can operate for up to 18 hours per day.  
Unit 2 is a newer and faster machine that produces 3,600 units per hour but can operate for at most 9 hours per day.

Note:
* The objective function is quadratic because it has only linear and squared terms. The quadratic terms model increasing operating costs due to wear, energy consumption, and inefficiencies at higher utilization levels.

* A quadratic function can also have bilinear ($x_1x_2$) terms which products of the two independent variables.

### Mathematical Formulation
#### Objective Function
This is the objective function that we want to minimize  
$\text{cost} = 0.4x_1^2 - 5x_1 + x_2^2 - 6x_2 + 50$

#### Constraints
